In [6]:
import numpy as np
from numpy import typing as npt
from numpy import random as rnd
from collections.abc import Callable, Sequence

In [7]:
def sigm(x: float):
    return 1.0 / (1 + np.exp(-x))

vsigm = np.vectorize(sigm)


In [8]:
def infer(input: npt.NDArray, nn: list[npt.NDArray]) -> list[npt.NDArray]:

    output = [input]

    for layer in range(len(nn)):
        input = np.hstack((1, output[layer]))
        output.append(vsigm(input @ nn[layer]))

    return output


In [9]:
def backprogation(train_input: npt.NDArray, train_output: npt.NDArray, topol: Sequence[int], iters: int, eta: float, seed: int=123):

    rng = rnd.default_rng(seed)

    # Create fully connected feed-forward network
    nn = []
    for i in range(len(topol) - 1):
        nn.append(rng.uniform(low=-0.05, high=0.05, size=(topol[i] + 1, topol[i + 1])))
    
    # Train!
    for _ in range(iters):

        for i in range(train_input.shape[0]):

            err = []

            output = infer(train_input[i, :], nn)

            err.insert(0, output[-1] * (1 - output[-1]) * (train_output[i, :] - output[-1]))

            for layer in range(len(nn) - 1, -1, -1):
                err.insert(0, output[layer] * (1 - output[layer]) * np.sum(nn[layer] * err))

            for layer in range(len(nn)):
                nn[layer] = nn[layer] + eta * err[layer] * output[layer]
        

    return nn

In [10]:
nn = backprogation(None, None, [3, 5, 2], 10, 0.01)



infer(np.array([1,4,-3]), nn)

[array([ 1,  4, -3]),
 array([0.50391261, 0.44213056, 0.53986958, 0.44922074, 0.52440987]),
 array([0.49955201, 0.49659934])]